# Mô tả bài toán:
**Xây dựng một thuật toán tự động đề xuất giá sản phẩm phù hợp khi đã được cung cấp các mô tả văn bản do người dùng nhập về sản phẩm của Mercari, bao gồm các chi tiết như tên danh mục sản phẩm, tên thương hiệu và tình trạng mặt hàng.**

# Import thư viện

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = 12,6
import warnings
warnings.filterwarnings('ignore')
seq_col_brew = sns.color_palette("YlGnBu_r", 4)
sns.set_palette(seq_col_brew)

In [ ]:
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# 1. Nhập Dữ Liệu

Giải nén file trên kaggle server

In [ ]:
!apt-get install p7zip
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/test.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/sample_submission.csv.7z

In [ ]:
!unzip /kaggle/input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
!unzip /kaggle/input/mercari-price-suggestion-challenge/test_stg2.tsv.zip

# 2. Phân Tích Dữ Liệu

In [ ]:
#Đọc dữ liệu
dataset = pd.read_table("train.tsv")

In [ ]:
#In kích thước của 2 tập train và test
dataset.head()

In [ ]:
x = dataset[['item_condition_id', 'shipping', 'brand_name', 'category_name']]
y = dataset['price']

In [ ]:
# Xem thông tin chi tiết về kiểu dữ liệu trong tập train
dataset.info(memory_usage='deep')

Có thể thấy dữ liệu có tổng cộng 8 cột và nhận thấy cột Category_name, brand_name and item_description có các giá trị 
Non-Null nhỏ hơn so với các cột còn lại
-> Các cột đó chứa các giá trị null
Vậy ta cần phải chuẩn hóa các giá trị null thành các giá trị có thể xử lý

In [ ]:
# Đếm các giá trị NaN trong khung dữ liệu 
x.isnull().sum()

In [ ]:
# Mảng của x
x.shape

In [ ]:
# Thay thế các giá trị NaN trong brand_name thành Other
x['brand_name'].fillna('Other', inplace=True)

In [ ]:
# Đếm các giá trị NaN trong khung dữ liệu  
x.isnull().sum()

brand_name đã về 0, vậy chúng ta đã xử lý thành công dữ liệu brand_name

In [ ]:
# Show category_name có giá trị NaN
dataset[dataset['category_name'].isnull()]

In [ ]:
# Với validation_split gây ra hiện tượng overfitting
# Vì thế chúng ta chọn sử dụng train_test_split() để thay thế
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.80, random_state = 0)

In [ ]:
x_train.info()

In [ ]:
x_train.shape

In [ ]:
x_train.isnull().sum()

In [ ]:
x_train.head()

# 3. Phân phối mục tiêu

Kiểm tra phân phối của biến Price

In [ ]:
# Biểu đồ của price
sns.distplot(y)

Có thể thấy được phân phối của biểu đồ đang bị lệch phải nên chúng ta sẽ lấy log() của đồ thị

In [ ]:
#Biểu đồ Logarit của Price
sns.distplot(np.log(y + 1))

In [ ]:
sns.distplot(y[y < 100])

Có thể thấy được tại những điểm (price % 5 = 0) thì tần suất của price tăng đột biến

In [ ]:
# Độ lệch chuẩn
y.std()

In [ ]:
# Xem sự phân phôi của item_condition_id
x_train.item_condition_id.hist()

In [ ]:
# Phí vận chuyển được trả bởi:
#    + người bán: 1 
#    + người mua: 0
x_train.shipping.hist()

In [ ]:
# Biểu đồ quan hệ giữa shipping và price
sns.barplot(x='shipping', y='price', data=dataset)

In [ ]:
dataset.describe(include=['O']) 

In [ ]:
# Thống kê tập train
dataset.describe()

In [ ]:
sns.barplot(x='item_condition_id', y='price', hue='shipping', data=dataset)

In [ ]:
sns.barplot(x='item_condition_id', y='price', data=dataset)

# 4.Category

chúng ta sẽ sử lý category do trong category sẽ có nhiều sub category

In [ ]:
# Cắt chuỗi
def category_split(category_name):
    try:
        return category_name.split('/')
    except:
        return ['Other', 'Other', 'Other']
x_train['Top_Level_Category'], x_train['Second_Level_Category'], x_train['Third_Level_Category'] = zip(*x_train['category_name'].apply(lambda x: category_split(x)))

In [ ]:
x_train.head(10)

In [ ]:
columns = ['Top_Level_Category', 'Second_Level_Category', 'Third_Level_Category']

In [ ]:
for col in columns:
   x_train[col].fillna('Other', inplace=True)

In [ ]:
x_train.isnull().sum()

In [ ]:
x_train.head(20)

In [ ]:
# Cắt cột categoly_name
x_train = x_train.drop('category_name', axis=1)

In [ ]:
x_train['brand_name'] = x_train['brand_name'].astype('category').cat.codes
x_train['Top_Level_Category'] = x_train['Top_Level_Category'].astype('category').cat.codes
x_train['Second_Level_Category'] = x_train['Second_Level_Category'].astype('category').cat.codes
x_train['Third_Level_Category'] = x_train['Third_Level_Category'].astype('category').cat.codes

In [ ]:
x_train.head()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

In [ ]:
linreg = LinearRegression(n_jobs=-1)
cv_scores = cross_val_score(linreg, x_train, y_train, cv=5, scoring='neg_mean_squared_error')
print(cv_scores.mean(), cv_scores.std())

In [ ]:
# đọc file sample_submission_stg2.csv
submission = pd.read_csv('sample_submission_stg2.csv')
pred = np.ones((submission.shape[0]))

In [ ]:
pred

In [ ]:
pred * y_train.mean()

In [ ]:
# gán pred mới
pred = pred * y_train.mean()

In [ ]:
# In mảng submission
submission.shape

In [ ]:
submission['price'] = pred

In [ ]:
# chuyển submission_df sang dạng .csv để submit
submission.to_csv('submission.csv', index=False)